# Gesture Recognition
In this project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, we read the folder names for training and validation. We also set the `batch_size` here. Note that we set the batch size in such a way that we are able to use the GPU in full capacity. We can keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 50 #experiment with the batch size

dim_x = 120
dim_y = 120

## Generator
This is one of the most important part of the code. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. We have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size, abalation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,30,2)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size    # calculate the number of batches
        if abalation is not None:
            num_batches = abalation//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),dim_x,dim_y,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = imresize(image, (dim_x ,dim_y, 3))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]-np.min(image[:,:,0]))/(np.max(image[:,:,0])-np.min(image[:,:,0]))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]-np.min(image[:,:,1]))/(np.max(image[:,:,1])-np.min(image[:,:,1]))   #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]-np.min(image[:,:,2]))/(np.max(image[:,:,2])-np.min(image[:,:,2]))   #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        #code for the remaining data points which are left after full batches
        if (len(folder_list)%batch_size) != 0:
            remaining_batch = len(folder_list)%batch_size
            if abalation is not None:
                remaining_batch = abalation%batch_size
            rem_batch_data = np.zeros((remaining_batch,len(img_idx),dim_x,dim_y,3))
            rem_batch_labels = np.zeros((remaining_batch,5))
            for folder in range(remaining_batch):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    image = imresize(image, (dim_x ,dim_y, 3))
                    rem_batch_data[folder,idx,:,:,0] = (image[:,:,0]-np.min(image[:,:,0]))/(np.max(image[:,:,0])-np.min(image[:,:,0]))   #normalise and feed in the image
                    rem_batch_data[folder,idx,:,:,1] = (image[:,:,1]-np.min(image[:,:,1]))/(np.max(image[:,:,1])-np.min(image[:,:,1]))   #normalise and feed in the image
                    rem_batch_data[folder,idx,:,:,2] = (image[:,:,2]-np.min(image[:,:,2]))/(np.max(image[:,:,2])-np.min(image[:,:,2]))   #normalise and feed in the image

                rem_batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
                yield rem_batch_data, rem_batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here we make the model using different functionalities that Keras provides. We will use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. The last layer is the softmax layer. We should design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
input_shape = (15, dim_x, dim_y, 3)

## Model 1

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.optimizers import Adam, SGD, RMSprop

#write your model here
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding = 'same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 3, 3)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 3, 3)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that we have written the model, the next step is to `compile` the model. When we print the `summary` of the model, we'll see the total number of parameters we have to train.

In [8]:
optimiser = SGD(lr=0.001,clipvalue=1.0) #optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 40, 40, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 40, 40, 64)     55360     
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 40, 40, 64)     256       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 3, 13, 13, 64)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32448)             0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
abalation = None
train_generator = generator(train_path, train_doc, batch_size, abalation=abalation)
val_generator = generator(val_path, val_doc, batch_size, abalation=abalation)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=2, min_lr=0.00001)  # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
# if (abalation%batch_size) == 0:
#     steps_per_epoch = int(abalation/batch_size)
# else:
#     steps_per_epoch = (abalation//batch_size) + 1

# if (abalation%batch_size) == 0:
#     validation_steps = int(abalation/batch_size)
# else:
#     validation_steps = (abalation//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, we'll be able to save the model at the end of each epoch.

In [13]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/train ; batch size = 50
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 30s - loss: 4.0056 - categorical_accuracy: 0.2667

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 198s 14s/step - loss: 3.5803 - categorical_accuracy: 0.2519 - val_loss: 1.3918 - val_categorical_accuracy: 0.3600

Epoch 00001: saving model to model_init_2020-12-1909_17_35.246486/model-00001-3.75091-0.26094-1.39178-0.36000.h5
Epoch 2/50
14/14 [==============================] - 21s 2s/step - loss: 0.5037 - categorical_accuracy: 0.8267 - val_loss: 1.8203 - val_categorical_accuracy: 0.3400

Epoch 00002: saving model to model_init_2020-12-1909_17_35.246486/model-00002-0.75791-0.71484-1.82034-0.34000.h5
Epoch 3/50
14/14 [==============================] - 70s 5s/step - loss: 1.0550 - categorical_accuracy: 0.5483 - val_loss: 1.2691 - val_categorical_accuracy: 0.4800

Epoch 00003: saving model to model_init_2020-12-1909_17_35.246486/model-00003-1.16397-0.51443-1.26913-0.48000.h5
Epoch 4/50
14/14 [==============================] - 33s 2s/step - loss: 0.3514 - categorical_accuracy: 0.8833 - val_loss: 1.0921 - val_categorical_accuracy: 0.5500

Epoch 0000

## Using Model 1 we are getting a validation categorical accuracy of around 0.6800 and the validation loss between 0.8 and 0.85.

## Model 2

### Now we will add dropout in layers after flatten layer in Model 2

In [14]:
curr_dt_time = datetime.datetime.now()

In [15]:
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding = 'same', input_shape=input_shape))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 3, 3)))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 3, 3)))

model_2.add(Flatten())
model_2.add(Dropout(0.5))
model_2.add(Dense(128, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [16]:
optimiser = Adam(lr=0.0005,clipvalue=1.0) #write your optimizer
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_3 (Batch (None, 15, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 7, 40, 40, 32)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 7, 40, 40, 64)     55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 40, 40, 64)     256       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 3, 13, 13, 64)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32448)             0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [17]:
abalation = None  #Set ablation to 100 for ablation runs.
train_generator = generator(train_path, train_doc, batch_size, abalation=abalation)
val_generator = generator(val_path, val_doc, batch_size, abalation=abalation)

In [18]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=2, min_lr=0.00001)  # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [19]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [20]:
# if (abalation%batch_size) == 0:
#     steps_per_epoch = int(abalation/batch_size)
# else:
#     steps_per_epoch = (abalation//batch_size) + 1

# if (abalation%batch_size) == 0:
#     validation_steps = int(abalation/batch_size)
# else:
#     validation_steps = (abalation//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [21]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size =Source path =  Project_data/train ; batch size = 50
 50
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 13s - loss: 10.2675 - categorical_accuracy: 0.2083

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 86s 6s/step - loss: 10.1853 - categorical_accuracy: 0.2055 - val_loss: 11.2682 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2020-12-1909_55_34.279723/model-00001-10.24915-0.20814-11.26821-0.27000.h5
Epoch 2/50
14/14 [==============================] - 18s 1s/step - loss: 6.2768 - categorical_accuracy: 0.4545 - val_loss: 8.8290 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to model_init_2020-12-1909_55_34.279723/model-00002-6.59614-0.42578-8.82896-0.22000.h5
Epoch 3/50
14/14 [==============================] - 59s 4s/step - loss: 6.7841 - categorical_accuracy: 0.3348 - val_loss: 8.7447 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2020-12-1909_55_34.279723/model-00003-7.15431-0.33956-8.74475-0.23000.h5
Epoch 4/50
14/14 [==============================] - 27s 2s/step - loss: 3.0359 - categorical_accuracy: 0.5934 - val_loss: 5.4500 - val_categorical_accuracy: 0.3200

Epoch 00

## Model 2 has a worse accuracy than Model 1, with validation loss ~1.25 and validation accuracy ~0.55

## Model 3

### Now we will build Model 3 with an extra CNN layer and no Dropouts.

In [30]:
curr_dt_time = datetime.datetime.now()

In [31]:
model_3 = Sequential()
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding = 'same', input_shape=input_shape))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 3, 3)))

model_3.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 3, 3)))

model_3.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_3.add(Flatten())
model_3.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [32]:
optimiser = RMSprop(lr=0.0005,clipvalue=1.0) #write your optimizer
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 7, 40, 40, 32)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 7, 40, 40, 64)     55360     
_________________________________________________________________
batch_normalization_6 (Batch (None, 7, 40, 40, 64)     256       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 3, 13, 13, 64)     0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 3, 13, 13, 128)    221312    
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [33]:
abalation = None
train_generator = generator(train_path, train_doc, batch_size, abalation=abalation)
val_generator = generator(val_path, val_doc, batch_size, abalation=abalation)

In [34]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=2, min_lr=0.00001)  # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [36]:
# if (abalation%batch_size) == 0:
#     steps_per_epoch = int(abalation/batch_size)
# else:
#     steps_per_epoch = (abalation//batch_size) + 1

# if (abalation%batch_size) == 0:
#     validation_steps = int(abalation/batch_size)
# else:
#     validation_steps = (abalation//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [37]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/train ; batch size =Epoch 1/50
 50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 13s - loss: 9.8173 - categorical_accuracy: 0.3100

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 87s 6s/step - loss: 9.8713 - categorical_accuracy: 0.3001 - val_loss: 10.3103 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2020-12-1910_45_29.721063/model-00001-9.86849-0.30769-10.31030-0.35000.h5
Epoch 2/50
14/14 [==============================] - 16s 1s/step - loss: 12.7565 - categorical_accuracy: 0.1819 - val_loss: 10.6264 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init_2020-12-1910_45_29.721063/model-00002-11.92675-0.22266-10.62641-0.33000.h5
Epoch 3/50
14/14 [==============================] - 61s 4s/step - loss: 6.7557 - categorical_accuracy: 0.3856 - val_loss: 8.2119 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-12-1910_45_29.721063/model-00003-7.36164-0.37691-8.21189-0.25000.h5
Epoch 4/50
14/14 [==============================] - 28s 2s/step - loss: 0.7726 - categorical_accuracy: 0.8091 - val_loss: 4.1941 - val_categorical_accuracy: 0.4000

Epoch 

## Model 3 has an validation loss ~0.9 and validation categorical accuracy of ~0.8.

## Model 4

### Now we will add two more layers of CNN for model 4.

In [38]:
curr_dt_time = datetime.datetime.now()

In [39]:
model_4 = Sequential()
model_4.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding = 'same', input_shape=input_shape))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_4.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_4.add(Flatten())
model_4.add(Dense(512, activation='relu'))
model_4.add(Dense(5, activation='softmax'))

In [40]:
optimiser = RMSprop(lr=0.0008,clipvalue=1.0) #write your optimizer
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_6 (Batch (None, 15, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 7, 60, 60, 32)     0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 7, 60, 60, 64)     55360     
_________________________________________________________________
batch_normalization_7 (Batch (None, 7, 60, 60, 64)     256       
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 3, 30, 30, 64)     0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 3, 30, 30, 128)    221312    
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [41]:
abalation = None
train_generator = generator(train_path, train_doc, batch_size, abalation=abalation)
val_generator = generator(val_path, val_doc, batch_size, abalation=abalation)

In [42]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=2, min_lr=0.00001)  # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [43]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [44]:
# if (abalation%batch_size) == 0:
#     steps_per_epoch = int(abalation/batch_size)
# else:
#     steps_per_epoch = (abalation//batch_size) + 1

# if (abalation%batch_size) == 0:
#     validation_steps = int(abalation/batch_size)
# else:
#     validation_steps = (abalation//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [45]:
model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/train ; batch size = 50
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 13s - loss: 12.1861 - categorical_accuracy: 0.1767

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 89s 6s/step - loss: 12.1290 - categorical_accuracy: 0.1739 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2020-12-1911_20_14.278737/model-00001-12.24373-0.17496-13.53920-0.16000.h5
Epoch 2/50
14/14 [==============================] - 19s 1s/step - loss: 12.2427 - categorical_accuracy: 0.2243 - val_loss: 13.5392 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2020-12-1911_20_14.278737/model-00002-12.46634-0.21484-13.53920-0.16000.h5
Epoch 3/50
14/14 [==============================] - 63s 4s/step - loss: 12.6050 - categorical_accuracy: 0.1598 - val_loss: 12.5721 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-12-1911_20_14.278737/model-00003-12.96692-0.16978-12.57211-0.22000.h5
Epoch 4/50
14/14 [==============================] - 28s 2s/step - loss: 5.5239 - categorical_accuracy: 0.4720 - val_loss: 5.5745 - val_categorical_accuracy: 0.2300



## Model 4 has a validation loss ~0.8 and validation catgorical accuracy of ~0.8.

## Model 5

### For Model 5 we will add dropout in the layers after flatten layer.

In [57]:
curr_dt_time = datetime.datetime.now()

In [58]:
model_5 = Sequential()
model_5.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding = 'same', input_shape=input_shape))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_5.add(BatchNormalization())
# model.add(Dropout(0.2))
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_5.add(BatchNormalization())
# model.add(Dropout(0.2))
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_5.add(BatchNormalization())
# model.add(Dropout(0.2))
model_5.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_5.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_5.add(BatchNormalization())
# model.add(Dropout(0.2))
model_5.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_5.add(Flatten())
model_5.add(Dropout(0.5))
model_5.add(Dense(512, activation='relu'))
model_5.add(Dropout(0.5))
model_5.add(Dense(5, activation='softmax'))

In [59]:
optimiser = RMSprop(lr=0.0005,clipvalue=1.0) #write your optimizer
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_6 (Batch (None, 15, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 7, 60, 60, 32)     0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 7, 60, 60, 64)     55360     
_________________________________________________________________
batch_normalization_7 (Batch (None, 7, 60, 60, 64)     256       
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 3, 30, 30, 64)     0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 3, 30, 30, 128)    221312    
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [60]:
abalation = None
train_generator = generator(train_path, train_doc, batch_size, abalation=abalation)
val_generator = generator(val_path, val_doc, batch_size, abalation=abalation)

In [61]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=2, min_lr=0.00001)  # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [62]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [63]:
# if (abalation%batch_size) == 0:
#     steps_per_epoch = int(abalation/batch_size)
# else:
#     steps_per_epoch = (abalation//batch_size) + 1

# if (abalation%batch_size) == 0:
#     validation_steps = int(abalation/batch_size)
# else:
#     validation_steps = (abalation//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [67]:
model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 10s 743ms/step - loss: 0.0262 - categorical_accuracy: 0.8881 - val_loss: 0.7159 - val_categorical_accuracy: 0.8100

Epoch 00001: saving model to model_init_2020-12-2006_50_15.615578/model-00001-0.03442-0.89954-0.71591-0.81000.h5
Epoch 2/50
14/14 [==============================] - 46s 3s/step - loss: 0.1239 - categorical_accuracy: 0.9278 - val_loss: 0.8857 - val_categorical_accuracy: 0.7200

Epoch 00002: saving model to model_init_2020-12-2006_50_15.615578/model-00002-0.13319-0.94089-0.88572-0.72000.h5
Epoch 3/50
14/14 [==============================] - 15s 1s/step - loss: 0.0683 - categorical_accuracy: 0.9761 - val_loss: 0.8587 - val_categorical_accuracy: 0.7900

Epoch 00003: saving model to model_init_2020-12-2006_50_15.615578/model-00003-0.08466-0.97270-0.85874-0.79000.h5
Epoch 4/50
14/14 [==============================] - 32s 2s/step - loss: 0.1280 - categorical_accuracy: 0.9231 - val_loss: 0.7658 - val_categorical_accuracy: 0.7400

Epoch 000

## Model 5 is the best model, it has a validation loss ~0.4. The validation categorical loss is ~0.9.

## Model 6

### For Model 6 we will add dropout in the 3D CNN Layers.

In [68]:
curr_dt_time = datetime.datetime.now()

In [69]:
model_6 = Sequential()
model_6.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', padding = 'same', input_shape=input_shape))
model_6.add(BatchNormalization())
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))
model_6.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_6.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', padding = 'same'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))
model_6.add(MaxPooling3D(pool_size=(1, 2, 2)))

model_6.add(Flatten())
model_6.add(Dropout(0.5))
model_6.add(Dense(512, activation='relu'))
model_6.add(Dropout(0.5))
model_6.add(Dense(5, activation='softmax'))

In [70]:
optimiser = RMSprop(lr=0.0005,clipvalue=1.0) #write your optimizer
model_6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_6.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 15, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_3 (Batch (None, 15, 120, 120, 32)  128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 7, 60, 60, 32)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 7, 60, 60, 64)     55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 60, 60, 64)     256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 60, 60, 64)     0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 3, 30, 30, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [71]:
abalation = None
train_generator = generator(train_path, train_doc, batch_size, abalation=abalation)
val_generator = generator(val_path, val_doc, batch_size, abalation=abalation)

In [72]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=2, min_lr=0.00001)  # write the REducelronplateau code here

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [73]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [74]:
# if (abalation%batch_size) == 0:
#     steps_per_epoch = int(abalation/batch_size)
# else:
#     steps_per_epoch = (abalation//batch_size) + 1

# if (abalation%batch_size) == 0:
#     validation_steps = int(abalation/batch_size)
# else:
#     validation_steps = (abalation//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [75]:
model_6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Epoch 1/50
Project_data/train ; batch size = 50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


12/14 [========================>.....] - ETA: 19s - loss: 4.8944 - categorical_accuracy: 0.2967

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


14/14 [==============================] - 128s 9s/step - loss: 4.6202 - categorical_accuracy: 0.3005 - val_loss: 6.0672 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2020-12-2112_08_23.269261/model-00001-4.72052-0.30015-6.06721-0.27000.h5
Epoch 2/50
14/14 [==============================] - 14s 982ms/step - loss: 0.8494 - categorical_accuracy: 0.7633 - val_loss: 5.4246 - val_categorical_accuracy: 0.4300

Epoch 00002: saving model to model_init_2020-12-2112_08_23.269261/model-00002-1.46100-0.65234-5.42460-0.43000.h5
Epoch 3/50
14/14 [==============================] - 42s 3s/step - loss: 1.7624 - categorical_accuracy: 0.4800 - val_loss: 5.4657 - val_categorical_accuracy: 0.2800

Epoch 00003: saving model to model_init_2020-12-2112_08_23.269261/model-00003-1.86472-0.47878-5.46574-0.28000.h5
Epoch 4/50
14/14 [==============================] - 21s 1s/step - loss: 0.7524 - categorical_accuracy: 0.8161 - val_loss: 5.0723 - val_categorical_accuracy: 0.3600

Epoch 00

## This model has a validation accuracy of ~0.75 and validation loss of ~1.1. The model is less effective than the previous model.

# Final Model:
## Model 5 is the final model. The model has validation accuracies ~0.9. The best epoch has a 0.92 accuracy.